# Dissertation Proof-of-Concept

A system capable of determining whether the user is looking at the top or the bottom of a screen.

## Webcam view in notebook 

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

In [ ]:
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle = display(None, display_id=True)

    while True:
        _, frame = cap.read()
        _, frame = cv2.imencode('.jpeg', frame)

        display_handle.update(Image(data=frame.tobytes()))

        if stopButton.value:
            cap.release()
            display_handle.update(None)

stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', 
    tooltip='Description',
    icon='square' 
)

thread = threading.Thread(target=view, args=(stopButton,))
display(stopButton)
thread.start()